<a href="https://colab.research.google.com/github/NataliaKolesnik/Car_Price_prediction_part2/blob/master/2_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
path_data = '/content/drive/MyDrive/SF_project_8/data/'

##Библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

from catboost import Pool, CatBoostRegressor

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

SEED = 73

## Загрузка и подготовка данных для обучения

In [ ]:
df_train = pd.read_csv(path_data+'train_for_ML.csv')
df_test = pd.read_csv(path_data+'test_for_ML.csv')

In [ ]:
# Списки с признаками

categorical_features = ['bodyType', 'brand', 'color_littel', 'fuelType_littel', 'model_info',
                        'vehicleTransmission', 'ПТС', 'Привод', ]

numerical_features = ['mileage', 'age_avto', 'engineDisplacement', 'enginePower_log', 'numberOfDoors', 'Владельцы']

target = 'price'
target_log = 'price_log'

In [ ]:
# Соберем датасеты

X = df_train[categorical_features + numerical_features]

X_test = df_test[categorical_features + numerical_features]

Y = df_train[target]
Y_log = df_train[target_log]

In [ ]:
# Стандартизируем числовые данные

scaler = MinMaxScaler()

X[numerical_features] = scaler.fit_transform(X[numerical_features])
X_test[numerical_features] = scaler.fit_transform(X_test[numerical_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [ ]:
X.describe()

,mileage,age_avto,engineDisplacement,enginePower_log,numberOfDoors,Владельцы
count,6682.00,6682.00,6682.00,6682.00,6682.00,6682.00
mean,0.12,0.21,0.42,0.54,0.76,0.55
std,0.08,0.13,0.16,0.18,0.29,0.42
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.06,0.11,0.30,0.42,0.67,0.00
50%,0.11,0.18,0.42,0.53,0.67,0.50
75%,0.17,0.29,0.45,0.66,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00


In [ ]:
X_test.describe()

,mileage,age_avto,engineDisplacement,enginePower_log,numberOfDoors,Владельцы
count,1671.00,1671.00,1671.00,1671.00,1671.00,1671.00
mean,0.12,0.21,0.42,0.44,0.75,0.55
std,0.08,0.14,0.16,0.19,0.30,0.41
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.06,0.11,0.30,0.31,0.67,0.00
50%,0.11,0.18,0.41,0.43,0.67,0.50
75%,0.17,0.29,0.45,0.57,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00


In [ ]:
# Разбиваем на обучающую и валидационную выборки

X_train, X_valid, Ylog_train, Ylog_valid = train_test_split(X, Y_log, test_size=0.25, random_state=SEED, shuffle=True)
X_train_, X_valid_, Y_train, Y_valid = train_test_split(X, Y, test_size=0.25, random_state=SEED, shuffle=True)

In [ ]:
# Готовим pool для catboost

train_pool = Pool(X_train, Y_train, cat_features=categorical_features)
valid_pool = Pool(X_valid, Y_valid, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

## Обучение модели

In [ ]:
# создаем модель
model = CatBoostRegressor(
    depth=8, 
    l2_leaf_reg=0.3,
    learning_rate=0.008, #0.03
    iterations=10000, 
    eval_metric='MAPE', 
    custom_metric=['RMSE', 'MAE'],
    loss_function= 'RMSE', #'MAE', 
    random_seed=SEED,
    # grow_policy='Depthwise',
    # score_function='L2'
    )


In [ ]:
# обучаем модель
model.fit(train_pool, 
          eval_set=valid_pool, 
          #logging_level='Silent', 
          verbose_eval=500,
          use_best_model=True,
          plot=False)

0:	learn: 0.9931037	test: 1.0068891	best: 1.0068891 (0)	total: 20.5ms	remaining: 3m 24s
500:	learn: 0.1787162	test: 0.1930234	best: 0.1930234 (500)	total: 9.07s	remaining: 2m 52s
1000:	learn: 0.1389697	test: 0.1599827	best: 0.1599827 (1000)	total: 18.2s	remaining: 2m 43s
1500:	learn: 0.1249105	test: 0.1514843	best: 0.1514843 (1500)	total: 30.1s	remaining: 2m 50s
2000:	learn: 0.1144667	test: 0.1468650	best: 0.1468650 (2000)	total: 39.7s	remaining: 2m 38s
2500:	learn: 0.1059380	test: 0.1436646	best: 0.1436646 (2500)	total: 49.5s	remaining: 2m 28s
3000:	learn: 0.0987554	test: 0.1411525	best: 0.1411525 (3000)	total: 59.7s	remaining: 2m 19s
3500:	learn: 0.0927551	test: 0.1395073	best: 0.1395044 (3499)	total: 1m 9s	remaining: 2m 9s
4000:	learn: 0.0879806	test: 0.1381463	best: 0.1381463 (4000)	total: 1m 19s	remaining: 1m 59s
4500:	learn: 0.0838124	test: 0.1371878	best: 0.1371690 (4485)	total: 1m 29s	remaining: 1m 49s
5000:	learn: 0.0804199	test: 0.1368856	best: 0.1368778 (4994)	total: 1m 39s	

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,mileage,21.37
1,age_avto,19.81
2,enginePower_log,16.76
3,engineDisplacement,10.50
4,bodyType,6.68
5,model_info,6.16
6,brand,5.68
7,fuelType_littel,4.02
8,Привод,2.93
9,color_littel,2.59


In [1]:
# model_gp = CatBoostRegressor(
#     iterations=1000, 
#     eval_metric='MAPE', 
#     )

# grid = {
#         'learning_rate': [0.03, 0.01, 0.003, 0.008],
#         'depth': [2, 4, 6, 8],
#         'l2_leaf_reg': [0.2, 0.5, 1, 3],
#         'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
#         'score_function': ['Cosine', 'L2', 'NewtonL2', 'NewtonCosine'],
#         }
# model_gp.grid_search(grid, train_pool, verbose=200)

In [ ]:
# Проверка предсказания на вилидационных данных
y_valid_predict = model.predict(valid_pool)
#y_valid_predict = np.exp(y_valid_predict) 
mean_absolute_percentage_error(Y_valid, y_valid_predict)

0.13537077663078892

## Предсказание

In [ ]:
# предсказываем результат 
y_test_predict = model.predict(test_pool)

In [ ]:
y_test_predict

array([ 565778.5005646 , 2509272.33828612,  678959.96100167, ...,
       1527683.56710237, 1019481.07571156, 1051410.09748759])

In [ ]:
#y_test_predict = np.exp(y_test_predict)

In [ ]:
submission = pd.read_csv(path_data+'sample_submission.csv')
submission['price'] = y_test_predict
submission.to_csv(path_data+'sample_submission_catboost.csv', index=False)

In [ ]:
submission

,sell_id,price
0,1099427284,540619.26
1,1096405886,2606521.54
2,1100195294,657704.32
3,1099827282,485396.01
4,1100076198,2254871.95
...,...,...
1666,1099414488,878076.05
1667,1099277058,2422482.65
1668,1098773964,1545783.40
1669,1098835932,994154.99
